# Обработка видео
Классификация действий по видео


## Переключение версии TensorFlow

In [ ]:
%tensorflow_version 2.x

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
import os               # Модуль для работы с файловой системой: создание, удаление, навигация по папкам и файлам
import glob             # Модуль для поиска файлов по шаблону, например, все файлы с расширением .jpg в папке
import random           # Модуль для работы со случайными числами и случайным выбором элементов из списка
import numpy as np      # Библиотека для числовых вычислений, работы с многомерными массивами и матрицами
import matplotlib.pyplot as plt  # Модуль для построения графиков и визуализации данных (plt — сокращение для удобства)
import tensorflow as tf # Фреймворк для машинного обучения и создания нейронных сетей; tf — часто используемый псевдоним


In [ ]:
print(tf.__version__)

2.12.0


In [ ]:
if 1:  # Условие всегда истинно, поэтому код внутри блока выполнится всегда
    !pip install scikit-video==1.1.11
    # Команда для установки библиотеки scikit-video версии 1.1.11 через пакетный менеджер pip.
    # В Jupyter или Colab символ '!' позволяет запускать команды терминала из Python.

import skvideo.io
# Импорт модуля io из библиотеки scikit-video, который используется для чтения и записи видеофайлов.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 6.7 MB/s eta 0:00:00


## Загрузка и распаковка датасета KTH

In [ ]:
if 1:
    # Условие всегда истинно, код внутри выполнится всегда.

    !wget http://www.nada.kth.se/cvap/actions/walking.zip
    # Команда для загрузки файла walking.zip с указанного URL через утилиту wget (скачивание из интернета).
    !wget http://www.nada.kth.se/cvap/actions/jogging.zip
    !wget http://www.nada.kth.se/cvap/actions/running.zip
    !wget http://www.nada.kth.se/cvap/actions/boxing.zip
    !wget http://www.nada.kth.se/cvap/actions/handwaving.zip
    !wget http://www.nada.kth.se/cvap/actions/handclapping.zip
    # Аналогично скачиваются другие архивы с видео различных действий.

if 1:
    # Опять же, условие всегда истинно, команды выполняются.

    !jar xvf walking.zip -d walking > /dev/null
    # Команда распаковки архива walking.zip с помощью утилиты jar.
    # 'xvf' — параметры: x (extract), v (verbose — подробный вывод), f (filename).
    # '-d walking' — распаковать в папку walking.
    # '> /dev/null' — перенаправить вывод в "никуда", чтобы не показывать список файлов.

    !unzip jogging.zip -d jogging > /dev/null
    !unzip running.zip -d running > /dev/null
    !unzip boxing.zip -d boxing > /dev/null
    !unzip handwaving.zip -d handwaving > /dev/null
    !unzip handclapping.zip -d handclapping > /dev/null
    # Аналогично распаковываются остальные архивы с помощью утилиты unzip в соответствующие папки,
    # при этом вывод команд скрыт с помощью '> /dev/null'.


--2023-07-25 23:10:46--  http://www.kth.se/cvap/actions/walking.zip
Resolving www.kth.se (www.kth.se)... 130.237.28.40, 2001:6b0:1:11c2::82ed:1c28
Connecting to www.kth.se (www.kth.se)|130.237.28.40|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.kth.se/cvap/actions/walking.zip [following]
--2023-07-25 23:10:47--  https://www.kth.se/cvap/actions/walking.zip
Connecting to www.kth.se (www.kth.se)|130.237.28.40|:443... connected.
HTTP request sent, awaiting response... 404 
2023-07-25 23:10:47 ERROR 404: (no description).

URL transformed to HTTPS due to an HSTS policy
--2023-07-25 23:10:48--  https://www.kth.se/cvap/actions/jogging.zip
Resolving www.kth.se (www.kth.se)... 130.237.28.40, 2001:6b0:1:11c2::82ed:1c28
Connecting to www.kth.se (www.kth.se)|130.237.28.40|:443... connected.
HTTP request sent, awaiting response... 404 
2023-07-25 23:10:48 ERROR 404: (no description).

URL transformed to HTTPS due to an HSTS policy
--2023-07-25

In [ ]:
from google.colab import drive
# Импорт модуля drive из пакета google.colab, который позволяет работать с Google Диском в среде Google Colab.

drive.mount('/content/drive')
# Монтирует (подключает) ваш Google Диск к виртуальной файловой системе Colab по пути '/content/drive'.
# После этого вы сможете читать и записывать файлы на ваш диск, используя этот путь.
# При выполнении появится окно авторизации, где нужно будет разрешить доступ.


In [ ]:
if 1:
    # Условие всегда истинно, значит команды внутри блока будут выполнены.

    !unzip walking.zip -d walking > /dev/null
    # Распаковывает архив walking.zip в папку walking.
    # Параметр '-d walking' указывает директорию для распаковки.
    # '> /dev/null' подавляет вывод распаковки, чтобы не засорять консоль.

    !unzip jogging.zip -d jogging > /dev/null
    !unzip running.zip -d running > /dev/null
    !unzip boxing.zip -d boxing > /dev/null
    !unzip handwaving.zip -d handwaving > /dev/null
    !unzip handclapping.zip -d handclapping > /dev/null
    # Аналогично распаковываются другие архивы в соответствующие папки, без вывода подробностей.


  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of walking.zip or
        walking.zip.zip, and cannot find walking.zip.ZIP, period.
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of jogging.zip or
        jogging.zip.zip, and cannot find jogging.zip.ZIP, period.
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last di

## Подготовка датасета для классификации

In [ ]:
classes = [
    'walking',
    'jogging',
    'running',
    'boxing',
    'handwaving',
    'handclapping',
]
# Список классов — типов действий, которые представлены в видео.
# Каждый класс — это строка с названием действия.

dataset = []
# Пустой список для хранения данных: каждый элемент — кортеж (путь к видео, индекс класса).

data_root = './'
# Корневая папка с данными — текущая директория (./ означает "здесь").

for cls in classes:
    # Цикл по каждому классу в списке classes.

    print('Processing class: {}'.format(cls))
    # Выводит сообщение о том, какой класс сейчас обрабатывается.

    for fpath in glob.glob(os.path.join(data_root, cls, '*.avi')):
        # glob.glob ищет все файлы с расширением .avi в папке data_root/cls
        # os.path.join соединяет части пути, учитывая особенности ОС.

        cls_idx = classes.index(cls)
        # Находит числовой индекс текущего класса в списке classes.
        # Например, 'walking' будет 0, 'jogging' — 1 и т.д.

        dataset.append((fpath, cls_idx))
        # Добавляет в список dataset кортеж: (путь к файлу видео, индекс класса).
        # Это удобно для обучения, чтобы знать, к какому классу относится видео.


Processing class: walking
Processing class: jogging
Processing class: running
Processing class: boxing
Processing class: handwaving
Processing class: handclapping


In [ ]:
SUBSET_LEN = 180
# Константа, определяющая длину подвыборки датасета — сколько видео файлов мы хотим использовать.

random.shuffle(dataset)
# Перемешивает список dataset случайным образом, чтобы данные не были упорядочены по классам или порядку загрузки.

dataset = dataset[:SUBSET_LEN]
# Оставляет только первые 180 элементов из перемешанного списка dataset — создает подвыборку.

print('Dataset samples (subset):', len(dataset))
# Выводит количество элементов в подвыборке dataset, чтобы проверить размер выборки.


Dataset samples (subset): 0


In [ ]:
dataset

[]

## Визуализация кадра из видео

In [ ]:
videodata = skvideo.io.vread(dataset[0][0])
# Читает видеофайл, путь к которому хранится в первом элементе списка dataset.
# vread загружает все кадры видео в виде 4-мерного массива (num_frames, height, width, channels).

videodata = videodata.astype(np.float32) / 255.
# Преобразует данные видео из целых чисел (обычно 0-255 для цвета) в числа с плавающей точкой типа float32.
# Делит на 255, чтобы нормализовать значения пикселей в диапазон от 0 до 1 — это удобно для нейронных сетей.

print('videodata shape:', videodata.shape)
# Выводит форму массива videodata.
# Обычно форма (количество кадров, высота, ширина, количество цветовых каналов).

plt.imshow(videodata[50, ...])
# Показывает 50-й кадр из видео (индексация с нуля).
# plt.imshow отображает изображение, '...' означает взять все остальные измерения для этого кадра.


IndexError: ignored

## Визуализация "движения"

In [ ]:
motion = np.mean(videodata[1:, ...] - videodata[:-1, ...], axis=3, keepdims=True)
# Вычисляет разницу между соседними кадрами видео (кадр i+1 минус кадр i), чтобы оценить движение.
# videodata[1:, ...] — все кадры, кроме первого.
# videodata[:-1, ...] — все кадры, кроме последнего.
# Разница показывает изменения по времени между кадрами.
# np.mean(..., axis=3, keepdims=True) усредняет разницу по цветовому каналу (ось 3 — RGB), оставляя размерность (num_frames-1, height, width, 1).
# keepdims=True сохраняет размерность канала для удобства.

print('motion shape:', motion.shape)
# Выводит размерность массива motion.
# Обычно (число кадров минус 1, высота, ширина, 1).

plt.imshow(motion[50, ..., 0])
# Отображает 50-й кадр массива motion.
# motion[50, ..., 0] — выбираем 50-й кадр, все пиксели и единственный канал (индекс 0).
# Это показывает движение на 50-м временном интервале между кадрами.


## Создание модели CNN

In [ ]:
model = tf.keras.Sequential([
    # Создание последовательной модели Keras — слои будут идти друг за другом.

    tf.keras.layers.Conv3D(32, (5, 5, 5), (1, 2, 2), padding='same', activation='relu'),
    # 3D сверточный слой с 32 фильтрами, каждый размером 5x5x5 (время, высота, ширина).
    # strides=(1, 2, 2) — шаг свертки: 1 по времени, 2 по высоте и ширине (уменьшает пространственные размеры).
    # padding='same' — добавляет паддинг, чтобы выход был того же размера по пространству.
    # activation='relu' — функция активации ReLU (обнуляет отрицательные значения).

    tf.keras.layers.MaxPool3D((1, 2, 2), padding='same'),
    # Макспулинг (максимальное объединение) по объему с окном 1x2x2.
    # Сохраняет временной размер, уменьшает пространственные размеры.
    # padding='same' — добавляет паддинг.

    tf.keras.layers.Conv3D(64, (5, 5, 5), (1, 2, 2), padding='same', activation='relu'),
    # Второй 3D сверточный слой с 64 фильтрами и теми же параметрами размера и шага.

    tf.keras.layers.MaxPool3D((1, 2, 2), padding='same'),
    # Макспулинг как выше.

    tf.keras.layers.Conv3D(64, (3, 3, 3), (1, 2, 2), padding='same', activation='relu'),
    # Третий 3D сверточный слой с меньшим ядром 3x3x3.

    tf.keras.layers.MaxPool3D((1, 2, 2), padding='same'),
    # Макспулинг.

    tf.keras.layers.Conv3D(64, (3, 3, 3), (1, 1, 1), padding='same', activation=None),
    # Четвертый 3D сверточный слой, шаг (1,1,1) — сохраняет размер, без активации (линейный слой).

    tf.keras.layers.GlobalAveragePooling3D(),
    # Глобальный средний пуллинг по всем трем пространственным измерениям (время, высота, ширина),
    # превращает 5D тензор в 2D (батч, количество фильтров).

    tf.keras.layers.Dense(64, activation='relu'),
    # Полносвязный слой (Dense) с 64 нейронами и ReLU активацией.

    tf.keras.layers.Dense(6, activation=None),
    # Выходной полносвязный слой с 6 нейронами — по числу классов.
    # activation=None — линейный выход, можно использовать для логитов (например, с последующей softmax).
])


In [ ]:
inp = motion[None, ...]
# Добавляет новую ось в начало массива motion, чтобы получить 5D тензор (1, frames, height, width, channels).
# Это нужно, потому что модель ожидает вход с размерностью (batch_size, time, height, width, channels).
# None — эквивалентно np.newaxis, расширяет размерность.

out = model(inp)
# Пропускает вход inp через модель, получая выход (логиты или предсказания).

print('Input shape:', inp.shape)
# Выводит форму входных данных.
# Ожидается что-то типа (1, число_кадров, высота, ширина, каналы).

print('Output shape:', out.shape)
# Выводит форму выходных данных модели.
# Обычно (1, 6) — 1 пример в батче, 6 чисел (по числу классов).


## Подготовка к обучению

In [ ]:
NUM_EPOCHS = 10
# Количество эпох — сколько раз модель пройдет по всему обучающему набору данных.

LEARNING_RATE = 0.001
# Скорость обучения — размер шага при обновлении весов модели во время обучения.

model.compile(
    # Подготовка модели к обучению: задаются функция потерь и оптимизатор.

    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    # Функция потерь: разреженная кросс-энтропия для многоклассовой классификации.
    # from_logits=True означает, что входы — необработанные логиты (без softmax).
    # Sparse — значит, метки классов — целые числа, а не one-hot векторы.

    optimizer=tf.keras.optimizers.Adam(LEARNING_RATE)
    # Оптимизатор Adam — адаптивный метод градиентного спуска с заданной скоростью обучения.
)

writer = tf.summary.create_file_writer('logs/exp1')
# Создает объект для записи логов TensorBoard в папку 'logs/exp1'.
# Это позволяет визуализировать процесс обучения, метрики и графики.


## Цикл обучения модели

In [ ]:
global_step = 0
# Глобальный счетчик шагов обучения (итераций), используется для логов.

for ep in range(NUM_EPOCHS):
    # Цикл по эпохам — полный проход по датасету заданное количество раз.

    for iter, (fpath, label) in enumerate(dataset):
        # Цикл по элементам датасета, где iter — индекс итерации,
        # fpath — путь к видео, label — числовой класс.

        videodata = skvideo.io.vread(fpath)
        # Чтение видео по пути fpath.

        videodata = videodata.astype(np.float32) / 255.
        # Преобразование к float и нормализация пикселей в [0,1].

        motion = np.mean(videodata[1:, ...] - videodata[:-1, ...], axis=3, keepdims=True)
        # Вычисление движения между соседними кадрами (усреднение по цвету).

        x = motion[None, ...]
        # Добавление измерения батча для подачи в модель.

        y = np.array(label)[None, ...]
        # Преобразование метки в массив с размерностью батча.

        loss_value = model.train_on_batch(x, y)
        # Один шаг обучения на батче x, y.
        # Возвращает значение функции потерь после обновления весов.

        if iter % 10 == 0:
            # Каждые 10 итераций выводим прогресс.

            print(f'[{ep}/{NUM_EPOCHS}][{iter}/{len(dataset)}] Loss = {loss_value}')
            # Выводим текущую эпоху, итерацию и значение потерь.

            with writer.as_default():
                # Открываем контекст записи логов для TensorBoard.

                tf.summary.scalar('loss', loss_value, global_step)
                # Записываем значение потерь в логи с текущим global_step.

        global_step += 1
        # Увеличиваем счетчик шагов обучения.


## TensorBoard

In [ ]:
%load_ext tensorboard
# Загружает расширение TensorBoard в Jupyter/Colab, чтобы можно было запускать визуализацию прямо в ноутбуке.

%tensorboard --logdir logs
# Запускает TensorBoard и указывает папку с логами ('logs'), где записаны данные обучения.
# Позволяет интерактивно смотреть графики потерь, метрики и другие данные обучения в браузере или ноутбуке.


## Тестирование обученной модели

In [ ]:
fpath, cls_true = random.choice(dataset)
# Случайно выбирает один элемент из датасета: путь к видео (fpath) и настоящий класс (cls_true).

videodata = skvideo.io.vread(fpath)
# Считывает видео по выбранному пути.

videodata = videodata.astype(np.float32) / 255.
# Преобразует видео в формат float32 и нормализует пиксели в диапазон [0,1].

plt.imshow(videodata[30, ...])
# Отображает 30-й кадр видео для визуальной проверки.

motion = np.mean(videodata[1:, ...] - videodata[:-1, ...], axis=3, keepdims=True)
# Вычисляет движение между соседними кадрами (усреднение по каналам цвета).

out = model(motion[None, ...])[0]
# Пропускает движение через модель, добавляя размер батча.
# Берет первый элемент результата (поскольку батч из одного примера).

cls_pred = np.argmax(out.numpy())
# Преобразует результат в numpy-массив и выбирает индекс класса с максимальным значением (предсказание).

print('True class:', classes[cls_true])
# Выводит настоящий класс видео.

print('Predicted class:', classes[cls_pred])
# Выводит класс, предсказанный моделью.
